In [2]:
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.preprocessing import MinMaxScaler

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/GonieAhn/Data-Science-online-course-from-gonie/main/Data%20Store/example_data.csv')

In [6]:
col = []
missing = []
level = []

for name in data.columns:
    misper = data[name].isnull().sum() / data.shape[0]
    missing.append(round(misper, 4))
    
    lel = data[name].dropna()
    level.append(len(list(set(lel))))
    
    col.append(name)

In [7]:
summary = pd.concat([pd.DataFrame(col, columns=['Columns']),
                     pd.DataFrame(missing, columns=['Missing Percentage']),
                     pd.DataFrame(level, columns=['Level'])], axis=1)

In [8]:
summary

,Columns,Missing Percentage,Level
0,censor,0.0,2
1,event,0.0,358
2,age,0.0,52
3,wtkg,0.0,312
4,hemo,0.0,2
5,homo,0.0,2
6,drugs,0.0,2
7,karnof,0.0,4
8,oprior,0.0,2
9,z30,0.0,2
